In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [3]:
s = Structure(test_datas[11])
print(s.display())

solver = BasicSolver(s)
# solver.idxes_need_to_solve

+-----+-----+-----+
|. . 6|4 . 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. . 4|. . 9|
|     |     |     |
|7 . 8|. . .|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|. . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. . 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+



In [5]:
# solver.check_idx_only(62)
# print(solver.ready)
# solver.check_idx_last_left(62)
# print(solver.ready)
# print(solver.check_scanned_drop('9'))
print(solver.ready)
print(solver.check_square_drop('8'))
print(solver.ready)
solver.update()
print(solver.ready)

[]
[4, 6, 7, 23, 25, 26, 33, 34, 54, 60, 67, 69, 72, 76, 77, 80]
combines:
combine: ({4, 6, 7}, set())
combine: ({4, 6, 7}, {8, 5, 7})
combine: ({4, 6, 7}, {6, 7})
combine: ({4, 6, 7}, set())
combine: ({4, 6, 7}, set())
combine: ({4, 6, 7}, {0, 6})
combine: ({4, 6, 7}, {4, 6})
combine: ({4, 6, 7}, {0, 8, 4, 5})
combine: (set(), {8, 5, 7})
combine: (set(), {6, 7})
combine: (set(), set())
combine: (set(), set())
combine: (set(), {0, 6})
combine: (set(), {4, 6})
combine: (set(), {0, 8, 4, 5})
combine: ({8, 5, 7}, {6, 7})
combine: ({8, 5, 7}, set())
combine: ({8, 5, 7}, set())
combine: ({8, 5, 7}, {0, 6})
combine: ({8, 5, 7}, {4, 6})
combine: ({8, 5, 7}, {0, 8, 4, 5})
combine: ({6, 7}, set())
combine: ({6, 7}, set())
combine: ({6, 7}, {0, 6})
combine: ({6, 7}, {4, 6})
combine: ({6, 7}, {0, 8, 4, 5})
combine: (set(), set())
combine: (set(), {0, 6})
combine: (set(), {4, 6})
combine: (set(), {0, 8, 4, 5})
combine: (set(), {0, 6})
combine: (set(), {4, 6})
combine: (set(), {0, 8, 4, 5})
combine

In [6]:
print(solver.display())

+-----+-----+-----+
|. . 6|4 8 9|. . 5|
|     |     |     |
|1 8 4|2 . .|9 6 3|
|     |     |     |
|5 . .|3 6 .|4 . .|
+-----+-----+-----+
|. . 3|. . 4|. . 9|
|     |     |     |
|7 . 8|. . .|3 4 2|
|     |     |     |
|4 . .|8 . .|5 . .|
+-----+-----+-----+
|8 . .|9 4 6|. 5 1|
|     |     |     |
|9 . 5|. 8 2|. 3 4|
|     |     |     |
|. 4 .|5 . .|2 9 .|
+-----+-----+-----+



In [8]:
solver.steps

[(49, '9')]

In [9]:
tmp = "A	 	 	6	4	 	9	 	 	5
B	1	8	4	2	 	 	9	6	3
C	5	 	 	3	6	 	4	 	 
D	 	 	3	 	 	4	 	 	9
E	7	 	8	 	 	 	3	4	2
F	4	 	 	8	 	 	5	 	 
G	?	×	×	9	4	6	 	5	1
H	9	×	5	 	 	2	 	3	4
I	?	4	×	5	 	 	2	9	 "

In [10]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

161


'.,.,1,.,4,7,.,.,2,.,7,4,.,3,2,9,.,.,2,8,3,.,5,.,.,.,.,4,5,9,3,2,6,7,8,1,8,1,2,.,7,.,.,.,9,7,3,6,.,.,8,2,4,5,.,.,7,.,8,.,1,2,3,.,2,5,.,.,.,.,9,.,.,.,8,2,6,.,5,7,.'